In [ ]:
import os
from os import listdir
import pandas as pd  


def test_git_inference_multiple_image(image_path, model_name, prefix):
    param = {}
    cp = []
    id = []
    if File.isfile(f'aux_data/models/{model_name}/parameter.yaml'):
        param = load_from_yaml_file(f'aux_data/models/{model_name}/parameter.yaml')

    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

    for image_name in os.listdir(image_path):
      id.append(image_name)
      multiple_image_path = image_path + image_name
      if isinstance(multiple_image_path, str):
          if (multiple_image_path.endswith(".jpg")):
              multiple_images = [multiple_image_path]

      # if it is more than 1 image, it is normally a video with multiple image
      # frames
      img = [load_image_by_pil(i) for i in multiple_images]

      transforms = get_image_transform(param)
      img = [transforms(i) for i in img]

      # model
      model = get_git_model(tokenizer, param)
      pretrained = f'output/{model_name}/snapshot/model.pt'
      checkpoint = torch_load(pretrained)['model']
      load_state_dict(model, checkpoint)
      model.cuda()
      model.eval()
      img = [i.unsqueeze(0).cuda() for i in img]

      # prefix
      max_text_len = 40
      prefix_encoding = tokenizer(prefix,
                                  padding='do_not_pad',
                                  truncation=True,
                                  add_special_tokens=False,
                                  max_length=max_text_len)
      payload = prefix_encoding['input_ids']
      if len(payload) > max_text_len - 2:
          payload = payload[-(max_text_len - 2):]
      input_ids = [tokenizer.cls_token_id] + payload

      with torch.no_grad():
          result = model({
              'image': img,
              'prefix': torch.tensor(input_ids).unsqueeze(0).cuda(),
          })
      cap = tokenizer.decode(result['predictions'][0].tolist(), skip_special_tokens=True)
      cp.append(cap)
      
    # dictionary of lists  
    dict = {'image_id': id, 'caption': cp}  
    df = pd.DataFrame(dict) 
    # saving the dataframe 
    df.to_csv('output.csv') 